In [ ]:
import polars as pl
import pandas as pd
from dotenv import load_dotenv
from api.internal_api import SynthetixAPI, get_db_config

load_dotenv()

# constants
PROTOCOL = "toros"
DATA_LOC = "~/repos/data/parquet-data/indexers"
db_config = get_db_config(streamlit=False)
api = SynthetixAPI(db_config, streamlit=False)


In [ ]:
# entry fee
df_pl = pl.scan_parquet(
    [
        f"{DATA_LOC}/raw/optimism_mainnet/{PROTOCOL}/**/*_event_entry_fee_minted.parquet",
    ],
    include_file_paths="file_location",
)
df = df_pl.collect()
df_entry_fee = df.to_pandas()

# exit fee
df_pl = pl.scan_parquet(
    [
        f"{DATA_LOC}/raw/optimism_mainnet/{PROTOCOL}/**/*_event_exit_fee_minted.parquet",
    ],
    include_file_paths="file_location",
)
df = df_pl.collect()
df_exit_fee = df.to_pandas()

# withdrawals
df_pl = pl.scan_parquet(
    [
        f"{DATA_LOC}/raw/optimism_mainnet/{PROTOCOL}/**/*_event_withdrawal_initiated.parquet",
    ],
    include_file_paths="file_location",
)
df = df_pl.collect()
df_wd_init = df.to_pandas()

df_pl = pl.scan_parquet(
    [
        f"{DATA_LOC}/raw/optimism_mainnet/{PROTOCOL}/**/*_event_withdrawal_completed.parquet",
    ],
    include_file_paths="file_location",
)
df = df_pl.collect()
df_wd_complete = df.to_pandas()

df_pl = pl.scan_parquet(
    [
        f"{DATA_LOC}/raw/optimism_mainnet/{PROTOCOL}/**/*_event_withdrawal.parquet",
    ],
    include_file_paths="file_location",
)
df = df_pl.collect()
df_wd = df.to_pandas()

# deposits
df_pl = pl.scan_parquet(
    [
        f"{DATA_LOC}/raw/optimism_mainnet/{PROTOCOL}/**/*_event_deposit.parquet",
    ],
    include_file_paths="file_location",
)
df = df_pl.collect()
df_deposit = df.to_pandas()

# manager fee
df_pl = pl.scan_parquet(
    [
        f"{DATA_LOC}/raw/optimism_mainnet/{PROTOCOL}/**/*_event_manager_fee_minted.parquet",
    ],
    include_file_paths="file_location",
)
df = df_pl.collect()
df_manager_fee = df.to_pandas()

In [ ]:
df_pl = pl.scan_parquet(
    [
        f"{DATA_LOC}/raw/optimism_mainnet/{PROTOCOL}/**/transaction.parquet",
    ],
    include_file_paths="file_location",
)
df = df_pl.collect()
df_tx = df.to_pandas()

df_tx

In [ ]:
df_wd_complete

In [ ]:
df_wd_init

In [ ]:
df_wd

In [ ]:
df_manager_fee

## analysis plan:

- get all unique withdrawal initiated and completed transaction hashes, with the depositor/withdrawer address
- get all unique deposit transaction hashes, with the depositor/withdrawer address
- combine all manager entry and exit fees into a single dataframe
- combine withdrawal and deposit dataframes
- join the address from the deposit/withdrawals to the fee dataframe to determine who paid those fees
- Summarize this by depositor


In [ ]:
df_entry_fee

In [ ]:
withdrawal_txs = pd.concat(
    [
        df_wd_init[["transactionHash", "depositor", "blockTimestamp"]].rename(
            columns={"depositor": "address"}
        ),
        df_wd_complete[["transactionHash", "depositor", "blockTimestamp"]].rename(
            columns={"depositor": "address"}
        ),
        df_wd[["transactionHash", "investor", "blockTimestamp"]].rename(
            columns={"investor": "address"}
        ),
    ]
)
withdrawal_txs = withdrawal_txs.drop_duplicates("transactionHash")

# Get unique deposit transactions with investor info
deposit_txs = df_deposit[["transactionHash", "investor", "blockTimestamp"]].rename(
    columns={"investor": "address"}
)
deposit_txs = deposit_txs.drop_duplicates("transactionHash")

# Combine all transactions into single dataframe
all_txs = pd.concat(
    [withdrawal_txs.assign(tx_type="withdrawal"), deposit_txs.assign(tx_type="deposit")]
)

# Combine entry and exit fees
df_manager_fee["fees"] = df_manager_fee["daoFee"].apply(
    lambda x: int(x) / 1e18
) + df_manager_fee["managerFee"].apply(lambda x: int(x) / 1e18)
df_entry_fee["fees"] = df_entry_fee["entryFeeAmount"].apply(lambda x: int(x) / 1e18)
df_exit_fee["fees"] = df_exit_fee["exitFeeAmount"].apply(lambda x: int(x) / 1e18)
all_fees = pd.concat(
    [
        df_entry_fee[["transactionHash", "fees", "file_location"]].assign(
            fee_type="entry"
        ),
        df_exit_fee[["transactionHash", "fees", "file_location"]].assign(
            fee_type="exit"
        ),
        df_manager_fee[["transactionHash", "fees", "file_location"]].assign(
            fee_type="manager"
        ),
    ]
)

# Join fees with transaction data to get user addresses
fees_with_users = pd.merge(all_fees, all_txs, on="transactionHash", how="left")
# fees_with_users["file_location"][0]
# returns: '/Users/Troy/repos/data/parquet-data/indexers/raw/optimism_mainnet/toros/0128343148-0128780000/btc_bear1_x_event_entry_fee_minted.parquet'
# parse into just `BTC_BEAR_1x`
fees_with_users["token"] = fees_with_users["file_location"].apply(
    lambda x: "".join(x.split("/")[-1].split(".")[0].split("_")[:2])
)

# Group by address and summarize fees
fee_summary = (
    fees_with_users.groupby(["token", "address"])
    .agg(
        {
            "fees": "sum",
            "transactionHash": "count",
        }
    )
    .round(6)
)

fee_summary = fee_summary.reset_index().sort_values("fees", ascending=False)
fee_summary = fee_summary.rename(columns={"transactionHash": "num_transactions"})

# Also calculate total fees per user
total_fees = (
    fees_with_users.groupby("address")
    .agg(
        {
            "fees": "sum",
            "transactionHash": "count",
        }
    )
    .round(6)
)

total_fees = total_fees.reset_index().sort_values("fees", ascending=False)
total_fees = total_fees.rename(columns={"transactionHash": "total_transactions"})

print("\nTop 10 users by total fees paid:")
print(total_fees.sort_values("fees", ascending=False).head(25))

print("\nFee breakdown by type for top users:")
print(
    fee_summary[fee_summary["address"].isin(total_fees.nlargest(25, "fees")["address"])]
)

In [ ]:
# exports
fee_summary.to_csv("fee_summary.csv", index=False)
total_fees.head(25).to_csv("total_fees.csv", index=False)

In [ ]:
# run a check. Get total fees from the fees table and compare to the sum of entry, exit, and manager fees
total_fees_check = (
    fees_with_users.groupby("fee_type").agg({"fees": "sum"}).reset_index()
)
print(total_fees_check)

# get total fees from the summary
total_fees_check = fee_summary.groupby("fee_type").agg({"fees": "sum"}).reset_index()
print(total_fees_check)

# get the total from the raw dataframes
print(df_entry_fee["entryFeeAmount"].apply(lambda x: int(x) / 1e18).sum())
print(df_exit_fee["exitFeeAmount"].apply(lambda x: int(x) / 1e18).sum())
print(df_manager_fee["daoFee"].apply(lambda x: int(x) / 1e18).sum())
print(df_manager_fee["managerFee"].apply(lambda x: int(x) / 1e18).sum())
